## Case Classes

To create Datasets in Scala, we defined a Scala case class.

A case class is a regular class that is:

These traits make it quite valuable for data analysis because it is quite easy to reason about a case class. Probably the most importatn feature is that case classes are immutable and allow for comparison by structure instead of value.

According to the Scala Documentation:
- Immutability frees you from needing to keep track of where and when things are mutated
- Comparison by value allows you compare instances as if they are primitive values - no more uncertainty regarding whether instances of a class is complared by balue or reference 
- Pattern matching simplifies branching logic, which leads to less bugs and more readable code

## Actions

While we can see the power of Datasets what's important to understand is that actions like collect, take, and count apply to whether we are using Datasets or DataFrames.

flights.take(2)

You'll also notice that when we actually go to access one of the case classes we have to do no type coercion, we simply specify the named attribute of the case class and get back, not just the expected value but the expected type as well.

## Transofmrations

Now transformations on Datasets ahre the same as those that we saw on DataFrame. Any transformation that you read about in this section is valid on a Dataset and we encourage you to look throught the specific sections on relevant aggregations or joins.

## Filtering

Let's look at a simple example by creating a simple function that accepts a Flight and retunrs a boolean value that describe whether or not the origin and destination are the same. This is not a UDF but a generic function.

We can now pass this function into the filter method specifying that for each row it should verify that this function returns true and in the process will filter our Dataset down accordingly.

As we saw above, this function does not need to execute in Spark code at all. Similar to our UDFs, we can use it and test it on data on our local machines before using it within Spark.

## When to use Datasets

You might ponder , if I am going to pay a performace penalty when I use Datasets, why should I use Datasets at all?

There are several reasons that are worth considering when you use Dataset API. One consideration is that operations that are invalid, say subtracting two String types, will fail at compilation time not at runtime because Datasets are strongly typed. If correctness and bulletproof code is your highest priority at the sacrifice of performance, this can be a great choice for you.

Another time you may want to use Datasets is when you would like to reuse a variety of transformations of entire rows between single node workloads and Spark workloads. If you have some experience with Scala, you may notice that Spark's APIs reflect those of Native Scala Sequence Types, but in a distributed fashion. If you define all of your data and transformations as accepting case classes it is trivial to reuse them for both distributed and local workloads. Additionally when you collect your DataFrame to local disk they will be of the correct class and type, sometimes making further manipulation easier.

It's also worth considering that you can use both DataFrames and Datasets using whatever is most convenient for you at the time. For instance one common pattern one of the authors uses it to write their core ETL workflow in DataFrames and then when finally collecting some data to the driver, creating a Dataset in order to do so.

Also, when we go to collect our data, say for more local manipulation, you will be able to get that as a sequence or array of that specific data type, not of a Spark row. This, just as operating on the case classes you define, can help you reuse code and logic in both distributed and non-distributed settings.

# Chapter 10. Low Level API Overview

## The Low Level APIs

In the previous section we presented Spark's Structured APIs which are what most users should be using regularly to manipulate their data. There are times where this high level manipulation will not fit the business or engineering problem you are trying to solve. In those cases you may need to use Spark's lower level APIs specifically the Resilient Distributed Dataset te SparkContext, and Shared variables like accumulators and braodcast variables.

These lower level APIs should be used for two core reasons:

1. If you need some functionality that you cannot find in the higher level APIs. For the most part this case should be the exception.
2. You need to maintain some legacy codebase that runs on RDDs.

## When to use the low level APIs?

If you are brand new to Spark, this is not the place to start. Start with the Structured APIs, you'll be more productive more quickly

If you are an advanced developer hoping to get the most out of Spark, we still recommend focusing on the Structured APIs. However there are sometimes when you may want to "drop down" to some of the lower level tools in order to complete your task. These tools give you more fine-grained control at the expense of preventing you from shooting yourself in the foot. You may need to drop down to these APIs in order to use some legacy code, implement some custom partitioner, leverage a Broadcast variable or an Accumulator.

## The SparkCofn

The SparkConf manages all of the configurations for our environment. We create one via the import below.

## The SparkContext

A SparkContext represents the connection to a Spark cluster, and can be used to create RDDs, accumulators and braodcast variable on that cluster.

Prior to the consolidation of the SparkSession, the entrance point to executing Spark code that we used in previous chapters, Spark had two different contexts. Spark had a SparkContext and a SQLContext. The former focused on more fine grained control of Spark's central abstractions while the latter focused on the higher level tools like SparkSQL. The creators of Spark, in the version two of Spark, combined the two APIs into the centralized SparkSession that we have today. With that being said, both of these APIs can still be found in Spark today. We access both of them through the SparkSession variable. It is important to note that you should never need to use the SQLContext and rarely need to use the SparkContext. Here's how we access it.

## Broadcast Variables

Broadcast variables are immutable constants that Spark can replicate across every node in the cluster from the driver node. The use case for doing this would be to replicate some non-trivialy sized constant around the cluster such that Spark does not have to serialize it in a function to every node itself. This is commonly referred to as a Map-Side Join and can provide immnense speed ups when used correctly. We will touch on the implementation and use cases in the Distributed Variables Chapter.

## Accumulators

Accumulators, in a sence, are the opposite of a Broadcast Variable. Instead of replicating an immutable variable to all the nodes in the cluster, Accumulators create a mutable variable that each executor can update accordingly. This allows you to update a raw variable from each partition in the dataset in a safe way while event visualizing the results alogn the way in the Spark UI. We will touch on the implementation and use cases in the Distributed variables Chapter.

# Chapter 11. Basic RDD Operations

## RDD Overview

Resilient Distributed Datasets (RDDs) are Spark's oldest and lowest levle abstraction made available to users. They were the primary API in the 1.X Series and are still available in 2.X, but are not commonly used by end users. An important fact to note, however, is that virtually all Spark code you run, where DataFrames or Datasets, compiles down to an RDD. The Spark UI, mentioned in later chapters, also describes things in terms of RDDs and therefore it behooves users to have at least a basic understanding of what an RDD is and how to use it. While many users forego RDDs because virtually all functionality they provide is available in Datasets and RDDs, users can still use RDDs if they are handling legacy code. A Resilient Distributed Dataset, the basic abstraction in Spark, represnets an immutable, partitioned collection of elements that can be operated on in parallel.

RDDs give the user complet control because every row in an RDD is a just a Java object. Therefore RDDs do not need to have a Schema defined, or frankly anything defined. This gives the user great power but also makes manipulating data much more manual as a user has to "reinvent the wheel" for whatever task they are hoping to achieve. For example, users have to make usre that their Java objects have an efficient memory representation. Users will also have to implement their own filtering and mapping functions even to perform simple tasks like compute and average. Spark SQL obviates the need for the vast majority of this kind of work and does so in a highly optimized and efficient manner.

Internally, each RDD is characterized by five main properties:

- A list of partitions
- A function for computing each split
- A list of dependencies on other RDDs
- Optionally, a Partitioner for key-value RDDs
- Optionally, a list of preferred locations to compute each split on

``` 
The Partitioner is probably one of the core motivations for why you might want to use RDDs in your code. Specifying your own custom partitioner can give you significant performance and stability improvements if used correctly. These are discussed towards the end of the enxt Chapter when we introduce Key-Value Pair RDDs
``` 


These properties determine all of Spark's ability to schedule and execute the user program. Different kinds of RDDs implement their own versions of each of the above properties allow you to define new data sources.

RDDs follow the exact same Spark programming paradigms that we discussed in earlier chapters. We define transformations, which evaluate lazily, and actions, which evalute eagerly, to manipulate data in a distributed fashion. The transformations and actions that we defined previously apply conceptually to RDDs however in a much lower level interface. There is no concept of a "row" in RDDs, individual records are actually raw Java/Scala/Python objects and we manipulate those manually instead of tapping into the repository of functions that we have in the structured APIs. This chapter will show exampels in Scala but the APIs are quite similar across languages and there are countless examples across the web.

## From Data Sources

While you can create RDDs from data sources or text files. It's often preferable to use the Data srouce APIs. RDDs do not have a notion of "Data Source APIs" like DataFrames do, they primarily define their dependency structures and lists of partitions. If you are reading from any sort of Structures or Semi-Structured data source, we recommend using the Data Source API. This is assumming that a data source connector already exists for your data source. You can also create RDDs from plain-text filesis quite simple.

## Manipulating RDDS

We manipulate RDDs in much the same way that manipulate DataFrames. As mentioned, the core difference being that we manipulating raw Java or Scala objects instead of Spark type.s There are also a dearth of "helper" methods or functions thatwe can draw upon to simple calcultations. We must define filter functions, map functions, and other manipulations manually instead of leveraging those that already exist like we do in the Structured APIs.

## RDD of Case Class VS Dataset

We noticed this question on the web and found it to be an interesting question. The difference bewteen RDDs of Case Classes and Datasets is that Datasets can still take advantage of the wealth of functions that the Structured APIs have to offer. Wiwth Datasets, we do not have to choose between only operating on JVM types or on Spark types, we can choose whatever is either easiest to do or most flexible. We het the bot hof best worlds.

# chapter 12. Advaned RDDs Operations

The previous chapter explored RDds, which are Spark's most stable API. This chpater will inlcude relevant exampels and point to the documentation for others. There is a wealth of information available about RDDs across the web and becuase the APIs have not changed for years, we will focus on the core concepts as opposed to just API examples.

Advaned RDD operations revolve around three main concepts:
- Advaned single RDD Partition Level Operations
- Aggregations and Key-VAlue RDDs
- Custom Partitioning
- RDD Joins

## Pipe RDDs to System Commands

The pipe method is probably one of the more interesting methods that Spark has. It allows you to return an RDD created by piping elements to a forked external process. The resulting RDD is computed by executing the given process once per partition. All elements of each input parititon are written to a process's stdin as lines of input separated by a newline. The resulting partition consists of the process's stdout output, with each line of stdout resulting in one element of the output partition. A process is invoked even for empty partitions.

p 357